In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def sample(d0, d1, n=32):
    x0 = d0.sample((n,))
    x1 = d1.sample((n,))
    y0 = torch.zeros((n, 1))
    y1 = torch.ones((n, 1))
    return torch.cat([x0, x1], 0), torch.cat([y0, y1], 0)

In [ ]:
mu0 = torch.zeros(2)
mu1 = torch.Tensor([2, 2])
d0 = torch.distributions.MultivariateNormal(mu0, torch.eye(2))
d1 = torch.distributions.MultivariateNormal(mu1, torch.eye(2))
print(d0.sample((5,)))
print(d1.sample((5,)))

In [ ]:
net = nn.Sequential(nn.Linear(2, 5), nn.ReLU(), nn.Linear(5, 1))
for p in net.parameters():
    print(p.data)
net_opt = optim.SGD(lr=1e-3, params=list(net.parameters()))

In [ ]:
log_freq = 500
for i in range(20000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1, 100000)
            out = F.sigmoid(net(x))
            loss = F.binary_cross_entropy(out, y)
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    net_opt.zero_grad()
    x, y = sample(d0, d1, 1024)
    
#     out = F.sigmoid(layer(x))
#     loss = F.binary_cross_entropy(out, y)
    
    out = net(x)
    loss = F.binary_cross_entropy_with_logits(out, y)
    
    loss.backward()
    net_opt.step()

In [ ]:
net